# **TPS - Oct 2021**

## **XGBoost & LightGBM & CatBoost Stacking**

### Thank you for visiting my notebook :)
### This notebook is for beginner like me **who wants to study stacking ensemble!**

#### **Stacking Ensemble** is a nice technique for forwarding you score.
#### As you can see below image, Stacking Ensemble needs some models for classification and meta-model for final prediction!

#### Here's what you need to do.
**Step1. Make your train, test data for training & prediction (Preprocessing)**

**Step2. Select some models for making stacking datasets!! (Train models and Making Datasets)**

**Step3. Select final model for meta-model!**

**Step4. With your meta-model, Train & Predict with stacking datasets ;)**

![Stacking Ensemble](http://rasbt.github.io/mlxtend/user_guide/classifier/StackingClassifier_files/stackingclassification_overview.png)

# **Import Library**

In [3]:
import gc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# **Load Data**

In [4]:
# train_data = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')

clx = pd.read_csv('../input/tpsoct2021/stacking_train_clx.csv')
clx2_B1 = pd.read_csv('../input/tpsoct2021/stacking_train_clx2_B1.csv')
clx2_B2 = pd.read_csv('../input/tpsoct2021/stacking_train_clx2_B2.csv')
clx2_B3 = pd.read_csv('../input/tpsoct2021/stacking_train_clx2_B3.csv')
clx3_fc1 = pd.read_csv('../input/tpsoct2021/stacking_train_clx3_fc1.csv')
clx3_fc2 = pd.read_csv('../input/tpsoct2021/stacking_train_clx3_fc2.csv')
nn1 = pd.read_csv('../input/tpsoct2021-nn/oof_NN1.csv')
nn2 = pd.read_csv('../input/tpsoct2021-nn/oof_NN2.csv')
nn3 = pd.read_csv('../input/tpsoct2021-nn/oof_NN3.csv')
target = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv', usecols=['target'])
# test_data = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
clx_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx.csv')
clx2_B1_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx2_B1.csv')
clx2_B2_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx2_B2.csv')
clx2_B3_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx2_B3.csv')
clx3_fc1_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx3_fc1.csv')
clx3_fc2_test = pd.read_csv('../input/tpsoct2021/stacking_test_clx3_fc2.csv')
nn1_test = pd.read_csv('../input/tpsoct2021-nn/submission_NN1.csv',usecols=['target'])
nn2_test = pd.read_csv('../input/tpsoct2021-nn/submission_NN2.csv',usecols=['target'])
nn3_test = pd.read_csv('../input/tpsoct2021-nn/submission_NN3.csv',usecols=['target'])

print("Data imported")

Data imported


In [5]:
target.head()

,target
0,1
1,1
2,1
3,1
4,1


In [6]:
train_data = pd.concat([clx, clx2_B1, clx2_B2, clx2_B3, clx3_fc1, clx3_fc2, nn1, nn2, nn3], axis=1)
del clx, clx2_B1, clx2_B2, clx2_B3, clx3_fc1, clx3_fc2, nn1, nn2, nn3
gc.collect()


0

In [7]:
train_data.head()

,cat1,lgbm1,xgb1,cat2_B1,lgbm2_B1,xgb2_B1,cat2_B2,lgbm2_B2,xgb2_B2,cat2_B3,...,xgb2_B3,cat3_fc1,lgbm3_fc1,xgb3_fc1,cat3_fc2,lgbm3_fc2,xgb3_fc2,NN1,NN2,NN3
0,0.636516,0.659099,0.654883,0.768058,0.755764,0.764824,0.554211,0.605594,0.626031,0.794097,...,0.778191,0.558595,0.603123,0.607533,0.603311,0.616937,0.593090,0.465069,0.438382,0.440719
1,0.162521,0.186299,0.212586,0.366222,0.394592,0.389743,0.140930,0.162203,0.161350,0.422175,...,0.378377,0.151585,0.169611,0.177332,0.211078,0.192040,0.191485,0.471271,0.354400,0.345115
2,0.844496,0.852618,0.838314,0.484043,0.553811,0.554177,0.825709,0.858642,0.858205,0.517371,...,0.555654,0.862844,0.862994,0.856937,0.869971,0.874981,0.873232,0.850723,0.865030,0.831066
3,0.439135,0.527092,0.547408,0.722231,0.743720,0.747319,0.507045,0.553381,0.576203,0.732503,...,0.738712,0.587314,0.565700,0.620190,0.463408,0.492358,0.472813,0.489519,0.581348,0.532480
4,0.842151,0.830427,0.850084,0.522609,0.468266,0.474799,0.823246,0.815355,0.827911,0.481159,...,0.531830,0.818003,0.794510,0.800700,0.877110,0.831257,0.860441,0.878740,0.878620,0.910610


In [8]:
nn1_test.rename(columns={'target':'NN1'},inplace=True)
nn2_test.rename(columns={'target':'NN2'},inplace=True)
nn3_test.rename(columns={'target':'NN3'},inplace=True)

In [9]:
test_data = pd.concat([clx_test, clx2_B1_test, clx2_B2_test, clx2_B3_test, clx3_fc1_test, clx3_fc2_test,nn1_test, nn2_test, nn3_test], axis=1)
del clx_test, clx2_B1_test, clx2_B2_test, clx2_B3_test, clx3_fc1_test, clx3_fc2_test,nn1_test, nn2_test, nn3_test
gc.collect()

46

In [10]:
test_data.head()

,cat1,lgbm1,xgb1,cat2_B1,lgbm2_B1,xgb2_B1,cat2_B2,lgbm2_B2,xgb2_B2,cat2_B3,...,xgb2_B3,cat3_fc1,lgbm3_fc1,xgb3_fc1,cat3_fc2,lgbm3_fc2,xgb3_fc2,NN1,NN2,NN3
0,0.743567,0.725492,0.773843,0.379822,0.382807,0.382681,0.724124,0.736472,0.751203,0.390824,...,0.385558,0.722077,0.726661,0.744193,0.644519,0.412591,0.372536,0.604470,0.636643,0.613098
1,0.257177,0.247080,0.236851,0.466345,0.463944,0.487700,0.266524,0.252382,0.234636,0.455833,...,0.427886,0.265878,0.258234,0.253721,0.294631,0.529993,0.502947,0.348088,0.360540,0.370865
2,0.905104,0.918153,0.908728,0.592032,0.603186,0.587983,0.938882,0.946836,0.943005,0.531833,...,0.535434,0.936350,0.951365,0.946484,0.897739,0.790795,0.784437,0.878787,0.912274,0.913245
3,0.845935,0.836258,0.824549,0.532147,0.575965,0.556360,0.825365,0.824783,0.815317,0.550670,...,0.580966,0.822680,0.827817,0.817863,0.776154,0.611829,0.566786,0.743707,0.712590,0.733588
4,0.262387,0.277417,0.249826,0.492827,0.471721,0.471770,0.276785,0.286744,0.261271,0.466032,...,0.457980,0.277331,0.274110,0.266399,0.331941,0.525243,0.527473,0.320587,0.327888,0.325372


|||||||||||||||||||||||||||||||||||||||||||||||||||||

||||||||||||||||||||||||||||||||||||

In [11]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [12]:
train = reduce_memory_usage(train_data)
test = reduce_memory_usage(test_data)
y = reduce_memory_usage(target)

Mem. usage decreased to 40.05 Mb (75.0% reduction)
Mem. usage decreased to 20.03 Mb (75.0% reduction)
Mem. usage decreased to 0.95 Mb (87.5% reduction)


## **Checking Missing Values**

In [ ]:
print(train.isna().sum(), test.isna().sum())

# Feature Generation

In [ ]:
# train['std'] = train.std(axis=1)
# # train['min'] = train.min(axis=1)
# # train['max'] = train.max(axis=1)

# test['std'] = test.std(axis=1)
# # test['min'] = test.min(axis=1)
# # test['max'] = test.max(axis=1)

In [ ]:
# y = train['target']
# train.drop(columns = ['id', 'target'], inplace = True)
# test.drop(columns = 'id', inplace = True)

# **Modeling**

### **Stacking Data Loader**

In [13]:
def Stacking_Data_Loader(model, model_name, train, y, test, fold):
    '''
    Put your train, test datasets and fold value!
    This function returns train, test datasets for stacking ensemble :)
    '''

    stk = StratifiedKFold(n_splits = fold, random_state = 42, shuffle = True)
    
    # Declaration Pred Datasets
    train_fold_pred = np.zeros((train.shape[0], 1))
    test_pred = np.zeros((test.shape[0], fold))
    
    for counter, (train_index, valid_index) in enumerate(stk.split(train, y)):
        x_train, y_train = train.iloc[train_index], y[train_index]
        x_valid, y_valid = train.iloc[valid_index], y[valid_index]

        print('------------ Fold', counter+1, 'Start! ------------')
        if model_name == 'cat':
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)])
        elif model_name == 'xgb':
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric = 'auc', verbose = 500, early_stopping_rounds = 200)
        elif model_name == 'lgbm':
            model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], eval_metric = 'auc', verbose = 500, early_stopping_rounds = 200)
        print('------------ Fold', counter+1, 'Done! ------------')
        
        train_fold_pred[valid_index, :] = model.predict_proba(x_valid)[:, 1].reshape(-1, 1)
        test_pred[:, counter] = model.predict_proba(test)[:, 1]
        
        del x_train, y_train, x_valid, y_valid
        gc.collect()
        
    test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1, 1)
    
    del test_pred
    gc.collect()
    
    print('Done!')
    
    return train_fold_pred, test_pred_mean

### **Modeling**

#### Model's HyperParameters
* LGBM Param : https://www.kaggle.com/hiro5299834/tps-oct-2021-single-lightgbm
* Cat Param : https://www.kaggle.com/ranjeetshrivastav/tps-oct-21-catboost
* xgb Param : https://www.kaggle.com/rahulchauhan3j/tps-oct-2021-xgboost-pipeline-with-optuna#Model-Fit-and-Submission

Thanks for Sharing!

### **Stacking**

* Making train, test prediction array!
* Concat 3 arrays in 1 dataset

In [ ]:
# cat_train, cat_test = Stacking_Data_Loader(cat, 'cat', train, y, test, 5)
# del cat
# gc.collect()

# lgbm_train, lgbm_test = Stacking_Data_Loader(lgbm, 'lgbm', train, y, test, 5)
# del lgbm
# gc.collect()

# xgb_train, xgb_test = Stacking_Data_Loader(xgb, 'xgb', train, y, test, 5)
# del xgb
# gc.collect()

### **Final Stacking Datasets!**

In [ ]:
# stack_x_train = np.concatenate((cat_train, lgbm_train, xgb_train), axis = 1)
# stack_x_test = np.concatenate((cat_test, lgbm_test, xgb_test), axis = 1)
# # stack_x_train = np.concatenate((lgbm_train, xgb_train), axis = 1)
# # stack_x_test = np.concatenate(( lgbm_test, xgb_test), axis = 1)
# del cat_train, lgbm_train, xgb_train, cat_test, lgbm_test, xgb_test
# gc.collect()

# stack_x_train

In [ ]:
# stack_x_train.shape

### **Save stack_x_train**

In [ ]:
# cols = ['cat1','lgbm1', 'xgb1']
# df_stack = pd.DataFrame(stack_x_train,columns = cols)
# df_stack.to_csv('stacking_train_clx.csv', index = 0)
# df_stack.head()

In [ ]:
# cols = ['cat1','lgbm1', 'xgb1']
# df_test = pd.DataFrame(stack_x_test,columns = cols)
# df_test.to_csv('stacking_train_clx.csv', index = 0)
# df_test.head()

## Level 2 Training

In [ ]:
# params = {'device_type': 'cpu',
#  'n_estimators': 10000,
#  'learning_rate': 0.10932948761384702,
#  'num_leaves': 780,
#  'max_depth': 7,
#  'min_data_in_leaf': 9200,
#  'lambda_l1': 20,
#  'lambda_l2': 5,
#  'min_gain_to_split': 8.857618869765401,
#  'bagging_fraction': 0.8,
#  'bagging_freq': 1,
#  'feature_fraction': 0.8}

# overfit = {'device': 'cpu',
# #               'gpu_platform_id': 0,
# #               'gpu_device_id': 0,
#               'num_threads': -1,
#          'objective': 'binary','metric' : 'auc',
#     'n_estimators': 100000,
#     'random_state': 69,
#     'learning_rate': 3e-3,
#          }

In [29]:
def lgbm_fit(params, train, test, y):
    stk = StratifiedKFold(n_splits = 5)

    test_pred_lo = 0
    fold = 1
    total_auc = 0

    stack_x_train = np.array(train)
    stack_x_test = np.array(test)
    y_n = y.target
    train_fold_pred = np.zeros((train.shape[0], 1))
    
    for train_index, valid_index in stk.split(stack_x_train, y_n):
        x_train, y_train = stack_x_train[train_index], y_n[train_index]
        x_valid, y_valid = stack_x_train[valid_index], y_n[valid_index]

    #     model = RandomForestClassifier(random_state=32)
        model = LGBMClassifier(**params)
        model.fit(x_train, y_train
                  ,eval_set=[(x_valid,y_valid)],
                 early_stopping_rounds=200,
                verbose=0
                 )
        train_fold_pred[valid_index, :] = model.predict_proba(x_valid)[:, 1].reshape(-1, 1)
        valid_pred_lo = model.predict_proba(x_valid)[:, 1]
        test_pred_lo += model.predict_proba(stack_x_test)[:, 1]
        auc = roc_auc_score(y_valid, valid_pred_lo)
        total_auc += auc / 5
        print('Fold', fold, 'AUC :', auc)
        fold += 1

    print('Total AUC score :', total_auc)
    return train_fold_pred, test_pred_lo, total_auc

In [31]:
def objective(trial,train=train, test=test, y=y):
    params = {
        'n_estimators':10000,
        'objective': 'binary','metric' : 'auc',
         # L2 正則化
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
        # L1 正則化
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
        # 弱学習木ごとに使う特徴量の割合
        # 0.5 だと全体のうち半分の特徴量を最初に選んで, その範囲内で木を成長させる
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', .5, 1.),
        # 学習データ全体のうち使用する割合
        # colsample とは反対に row 方向にサンプルする
        'subsample': trial.suggest_loguniform('subsample', .5, 1.),
        # 木の最大の深さ
        # たとえば 5 の時各弱学習木のぶん機は最大でも5に制限される.
        'max_depth': trial.suggest_int('max_depth', low=3, high=8),
        # 末端ノードに含まれる最小のサンプル数
        # これを下回るような分割は作れなくなるため, 大きく設定するとより全体の傾向でしか分割ができなくなる
        # [NOTE]: 数であるのでデータセットの大きさ依存であることに注意
        'min_child_weight': trial.suggest_uniform('min_child_weight', low=.5, high=40)
    }

    train_fold_pred, test_pred_lo, total_auc = lgbm_fit(params, train,test, y)
    
    return total_auc

In [32]:
import optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

[I 2021-11-03 08:08:53,947] A new study created in memory with name: no-name-b68b9048-dbac-4e22-a32b-ddde60075040


Fold 1 AUC : 0.8571193576636036
Fold 2 AUC : 0.8597332072729746
Fold 3 AUC : 0.8597923742786449
Fold 4 AUC : 0.8553184931691702


[I 2021-11-03 08:10:16,824] Trial 0 finished with value: 0.8574841470468338 and parameters: {'reg_lambda': 0.07631423798351586, 'reg_alpha': 0.014470650146692647, 'colsample_bytree': 0.6765694856805176, 'subsample': 0.5801035252786808, 'max_depth': 4, 'min_child_weight': 15.686175133278997}. Best is trial 0 with value: 0.8574841470468338.


Fold 5 AUC : 0.8554573028497763
Total AUC score : 0.8574841470468338
Fold 1 AUC : 0.8571026567478897
Fold 2 AUC : 0.8597178245585011
Fold 3 AUC : 0.8597662279040438
Fold 4 AUC : 0.8552786081316425


[I 2021-11-03 08:11:36,129] Trial 1 finished with value: 0.8574530944876165 and parameters: {'reg_lambda': 0.008777800528617728, 'reg_alpha': 225.40259052723513, 'colsample_bytree': 0.6015057561068612, 'subsample': 0.5095404350801592, 'max_depth': 3, 'min_child_weight': 38.24075472503379}. Best is trial 0 with value: 0.8574841470468338.


...

Fold 5 AUC : 0.8554411580345856
Total AUC score : 0.8574928764350475
Fold 1 AUC : 0.857126355420188
Fold 2 AUC : 0.8597636068015777
Fold 3 AUC : 0.8598156458505412
Fold 4 AUC : 0.8553102260613916


[I 2021-11-03 10:38:18,274] Trial 98 finished with value: 0.8574903314126527 and parameters: {'reg_lambda': 2.830585706185732, 'reg_alpha': 15.327212057523058, 'colsample_bytree': 0.922056700889357, 'subsample': 0.7478219134179673, 'max_depth': 5, 'min_child_weight': 17.95293700683135}. Best is trial 83 with value: 0.8574964710784296.


Fold 5 AUC : 0.8554358229295658
Total AUC score : 0.8574903314126527
Fold 1 AUC : 0.8571318181253277
Fold 2 AUC : 0.8597518298404967
Fold 3 AUC : 0.8598239548583592
Fold 4 AUC : 0.8553156194664663


[I 2021-11-03 10:39:51,047] Trial 99 finished with value: 0.8574882849607273 and parameters: {'reg_lambda': 15.094697912902477, 'reg_alpha': 2.7972691491393373, 'colsample_bytree': 0.9735350851390586, 'subsample': 0.6481179350406446, 'max_depth': 6, 'min_child_weight': 36.916143249248705}. Best is trial 83 with value: 0.8574964710784296.


Fold 5 AUC : 0.8554182025129867
Total AUC score : 0.8574882849607273


In [33]:
print('Number of finished trials:', len(study.trials))
print('Best trial parameters:', study.best_trial.params)
print('Best score:', study.best_value)

Number of finished trials: 100
Best trial parameters: {'reg_lambda': 1.0218038207092495, 'reg_alpha': 44.60016377867773, 'colsample_bytree': 0.9992809773810531, 'subsample': 0.9496830721998346, 'max_depth': 5, 'min_child_weight': 38.29142371660059}
Best score: 0.8574964710784296


In [34]:
study_params = study.best_params

study_params

{'reg_lambda': 1.0218038207092495,
 'reg_alpha': 44.60016377867773,
 'colsample_bytree': 0.9992809773810531,
 'subsample': 0.9496830721998346,
 'max_depth': 5,
 'min_child_weight': 38.29142371660059}

In [35]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances

plot_optimization_history(study)

In [36]:
plot_param_importances(study)

In [37]:
best_cols = ['NN3', 'cat1', 'NN1', 'xgb2_B3', 'cat2_B3', 'cat2_B1', 'lgbm1']

train_pred, test_pred, auc= lgbm_fit(study_params, train[best_cols], test[best_cols], y)

Fold 1 AUC : 0.8571377208308815
Fold 2 AUC : 0.8597699753575698
Fold 3 AUC : 0.859734682674363
Fold 4 AUC : 0.8553191822198185
Fold 5 AUC : 0.8555023606921712
Total AUC score : 0.8574927843549608


# **Submission!**

## Blending Level 2 results

In [40]:
ver = "003"

In [41]:
cols = [f'model1_{ver}']
# stack_x_train = np.stack((train_pred_array[0], train_pred_array[1], train_pred_array[2]), 1)
# stack_x_test = np.stack((test_pred_array[0], test_pred_array[1], test_pred_array[2]), 1)
df_train_pred = pd.DataFrame(train_pred, columns = cols)
df_test_pred = pd.DataFrame(test_pred, columns = cols)

df_train_pred.to_csv(f'lgbm_train{ver}.csv', index=0)
df_test_pred.to_csv(f'lgbm_test{ver}.csv', index=0)
sub = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')
sub['target'] = test_pred
sub.to_csv('sub_lgbm.csv', index = 0)

sub.head()

,id,target
0,1000000,3.467406
1,1000001,1.422645
2,1000002,4.645026
3,1000003,4.057500
4,1000004,1.527712


# Done!


## If you think this notebook is helpful for you, Please do not forget upvote!